# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
gmaps_df = pd.read_csv('output_data/cities.csv')
gmaps_df.head()

,Unnamed: 0,city,cloudiness,humidity,latitude,longitude,max_temperature,wind_speed,country,datetime
0,0,provideniya,90,81,64.38,-173.30,50.00,11.18,RU,1595302095
1,1,chokurdakh,55,31,70.63,147.92,83.53,14.41,RU,1595302300
2,2,sao joao da barra,0,90,-21.64,-41.05,70.48,10.20,BR,1595302300
3,3,bowen,98,46,-20.02,148.23,73.40,12.75,AU,1595302300
4,4,punta arenas,3,92,-53.15,-70.92,30.20,6.93,CL,1595302300


In [3]:
#drops index column from prior dataframe
clean_gmaps_df = gmaps_df.drop(gmaps_df.columns[[0]], axis=1)

In [4]:
clean_gmaps_df

,city,cloudiness,humidity,latitude,longitude,max_temperature,wind_speed,country,datetime
0,provideniya,90,81,64.38,-173.30,50.00,11.18,RU,1595302095
1,chokurdakh,55,31,70.63,147.92,83.53,14.41,RU,1595302300
2,sao joao da barra,0,90,-21.64,-41.05,70.48,10.20,BR,1595302300
3,bowen,98,46,-20.02,148.23,73.40,12.75,AU,1595302300
4,punta arenas,3,92,-53.15,-70.92,30.20,6.93,CL,1595302300
...,...,...,...,...,...,...,...,...,...
572,northampton,25,91,40.88,-75.25,75.99,5.82,US,1595302375
573,sao felix do xingu,90,66,-6.64,-51.99,69.69,2.04,BR,1595302376
574,broken hill,92,54,-31.95,141.43,53.60,14.99,AU,1595302376
575,alihe,0,26,50.57,123.72,77.79,14.14,CN,1595302376


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#determine location data from df
locations = clean_gmaps_df[["latitude", "longitude"]]

#determine humidty
humidity = clean_gmaps_df["humidity"]

# Plot Heatmap
fig = gmaps.Map()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Map(configuration={'api_key': None}, data_bounds=[(-46.33451533539444, -179.99999), (85.0, 179.99999)], layers…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal weather conditions:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [13]:
# zero cloudiness
no_cloud = clean_gmaps_df[clean_gmaps_df["cloudiness"] == 0]
# max temp above 70 degrees Fahrenheit
above_70_degrees = clean_gmaps_df[clean_gmaps_df["max_temperature"] > 70]
# max temp below 80 degrees Fahrenheit
below_80_degrees = above_70_degrees[above_70_degrees["max_temperature"] < 80]
# wind speed below 10 mph
slow_wind = below_80_degrees[below_80_degrees["wind_speed"] < 10]
# humidity below 60 percent
ideal_vacation = slow_wind[slow_wind["humidity"] < 60]
# reset index to create ideal vacation df
ideal_vacation_df = ideal_vacation.reset_index()

In [14]:
ideal_vacation_df

,index,city,cloudiness,humidity,latitude,longitude,max_temperature,wind_speed,country,datetime
0,13,kodiak,40,46,57.79,-152.41,73.40,9.17,US,1595302301
1,42,xambioa,27,52,-6.41,-48.54,72.19,1.30,BR,1595302305
2,59,laguna,1,53,38.42,-121.42,75.00,6.93,US,1595302057
3,67,sao raimundo nonato,69,55,-9.02,-42.70,70.11,5.39,BR,1595302308
4,93,cherskiy,97,27,68.75,161.30,77.47,5.48,RU,1595302311
5,145,bilibino,16,26,68.05,166.44,75.96,4.74,RU,1595302318
6,184,puerto quijarro,0,48,-17.78,-57.77,71.42,2.95,BO,1595302324
7,197,batagay-alyta,100,55,67.80,130.41,78.10,7.49,RU,1595302325
8,226,airai,99,46,-8.93,125.41,73.81,4.16,TL,1595302329
9,271,benjamin aceval,20,57,-24.97,-57.57,75.20,6.93,PY,1595302335


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Create a hotel_df
starter_hotel_df = ideal_vacation_df.loc[:,["city","country", "latitude", "longitude"]]

# Add a "Hotel Name" column to the DataFrame.
starter_hotel_df["Hotel Name"] = ""

#Rename columns to match code below
mapping = {starter_hotel_df.columns[0]:'City', starter_hotel_df.columns[1]: 'Country', starter_hotel_df.columns[2]:'Lat',
           starter_hotel_df.columns[3]:'Lng'}
hotel_df_full = starter_hotel_df.rename(columns=mapping)

# limit to ten or less cities, depending on data retrieved
hotel_df = hotel_df_full.truncate(after=9)

#Display results
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,kodiak,US,57.79,-152.41,
1,xambioa,BR,-6.41,-48.54,
2,laguna,US,38.42,-121.42,
3,sao raimundo nonato,BR,-9.02,-42.70,
4,cherskiy,RU,68.75,161.30,
5,bilibino,RU,68.05,166.44,
6,puerto quijarro,BO,-17.78,-57.77,
7,batagay-alyta,RU,67.80,130.41,
8,airai,TL,-8.93,125.41,
9,benjamin aceval,PY,-24.97,-57.57,


In [16]:
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

parameters = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [17]:
#header
print("Beginning Data Retrieval     ")
for index, row in hotel_df.iterrows():
    #space
    print("-----------------------------")
    try:
        # retrieve latitude, longitude, city name
        hotel_lat = row["Lat"]
        hotel_lng = row["Lng"]
        city_name = row["City"]
        # location parameters determined by lat and lng
        parameters["location"] = f"{hotel_lat},{hotel_lng}"
        # API request
        print(f"Processing City # {index}: {city_name}.")
        hotel_response = requests.get(hotel_url, params=parameters).json()
        # results
        results = hotel_response['results']
        #print data
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    # exceptions
    except (KeyError, IndexError):
        print("Hotel not found, skip")

# indicate loading is finished
print("Data Retrieval Complete")

Beginning Data Retrieval     
-----------------------------
Processing City # 0: kodiak.
Closest hotel in kodiak is Kodiak Compass Suites.
-----------------------------
Processing City # 1: xambioa.
Closest hotel in xambioa is Hotel Rio Araguaia.
-----------------------------
Processing City # 2: laguna.
Closest hotel in laguna is Hilton Garden Inn Sacramento Elk Grove.
-----------------------------
Processing City # 3: sao raimundo nonato.
Closest hotel in sao raimundo nonato is Real Hotel.
-----------------------------
Processing City # 4: cherskiy.
Hotel not found, skip
-----------------------------
Processing City # 5: bilibino.
Closest hotel in bilibino is Profilaktoriy Baes.
-----------------------------
Processing City # 6: puerto quijarro.
Hotel not found, skip
-----------------------------
Processing City # 7: batagay-alyta.
Hotel not found, skip
-----------------------------
Processing City # 8: airai.
Hotel not found, skip
-----------------------------
Processing City # 9: b

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Map(configuration={'api_key': None}, data_bounds=[(-52.656940488224315, -180.0), (85.0, -180.0)], layers=(Weig…